In [8]:
# Load modules
import os
import pandas as pd
import re
import ast
import glob
import csv

import shutil
import numpy as np
import geopy
import pygmt
import pyproj
from geopy.distance import geodesic
from geopy.distance import distance
from os import system as sys

path = "C:/Users/19049/coding_files/data/CFM5.3_traces.ts"
subset = "C:/Users/19049/coding_files/data/subset.ts"
folder = "C:/Users/19049/coding_files/faults/"
wb_2b = "C:/Users/19049/coding_files/wb_2b/"

to_open = path

#currently the path is saevd as a .ts file, it'd be nice to have a conversion automatically from utm

In [ ]:
#I did not write this https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder
#Just a tool to clear the faults folder- but could be useful later when reading in new data
# def Delete_faults_content():
#     [f.unlink() for f in Path("C:/Users/19049/coding_files/faults").glob("*") if f.is_file()]
# Delete_faults_content()
# def Delete_wb_2b_content():
#     [f.unlink() for f in Path("C:/Users/19049/coding_files/wb_2b").glob("*") if f.is_file()]
# Delete_wb_2b_content()


for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
        
for filename in os.listdir(wb_2b):
    file_path = os.path.join(wb_2b, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [ ]:
# for filename in os.listdir(folder):
#     os.system(folder + 'rm *')

In [9]:
# @title
#what i've got:
#strip the crap from the fault name line (the tag and excess quotes, can expand to more stuff)
#open a fault folder to stick the coords in

#What i want:
#how tf do I put the coords into the new files now- I think I can use the current name and use the make_new_fault as a ticker, so the "put number in folder" function is dumb and every time it sees
#a coord it will just add it to the fault that is currently "name" and that should change since the for loop goes down the list in order
#it'd be nice to get a counter as a discrepancy check

#strips "> " and extra "" from line and sets it to global name
#change the calling of line to a thing within the function itself instead of depending on 'line' having the same name in the for loop
def redefine_name():

    t1_name = line
    t2_name = re.sub("[> ]", "", t1_name)
    name = ast.literal_eval(t2_name)

#makes an empty pandas dataframe using the empty list l
def make_new_fault():
    global l1
    l1 = pd.DataFrame(l,columns = ['Easting(m)','Northing(m)','Elevation(m)'])

#the line being read is delimited and turned from a line of strings into a line of floats
#then it is apended to the l1 dataframe
def number_line():
    string_line = line.split()
    float_list = list(map(float, string_line))
    l1.loc[len(l1)] = float_list

#takes the finished l1 dataframe and names it the current fault trace, making it a csv    
def save_line():
    l1.to_csv(folder+thing+".csv")
    
#what we want to do is move the current fault in names, to a folder by providing the current path and the path to the folder, also renames!
def move():
    shutil.move(folder+fault, wb_2b+short+'.csv')

In [15]:
print(l1)

     Easting(m)  Northing(m)  Elevation(m)
0   407001.1563   3906157.50         760.0
1   406313.9375   3907289.00         750.0
2   406092.1875   3908817.75         792.0
3   405931.8750   3910852.75         905.0
4   405538.5313   3912022.50        1010.0
..          ...          ...           ...
69  408141.0000   4024031.50        1131.0
70  407937.2187   4025852.50        1142.0
71  407708.6562   4027649.50        1157.0
72  407413.0000   4029714.75        1141.0
73  402188.7500   4040125.25        1644.0

[74 rows x 3 columns]


In [10]:
#13 faults that are included in the project that we want to make worlbuilder files of
# format: '*_fault',\n
faults_to_include = [
    'Panamint_Valley_fault',
    'Southern_Death_Valley_fault',
    'Black_Mountain_fault',
    'Owl_Lake_fault',
    'Hunter_Mountain_fault',
    'Ash_Hill_fault',
    'Independence_Sierra_Nevada_fault', #Gotta add the _fault identification bit cause of
    'Independence_fault',               #these two lines
    'Little_Lake_fault',
    'Airport_Lake_fault',
    'Eastern_Little_Lake_north_north_east_cross_fault',
    'Owens_Valley_fault',
    'Northern_Death_Valley_fault'
    ]

so, as the reader is looking through each line, instead of writing out the whole thing we want it to aknowledge that the line has the want in it
we can do that by 

In [68]:
print(flag)

['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']


In [14]:
#l being here resets the dataframe every time it's run, though I can probably just define make_new_fault pd.dataframe([],columns....) instead of making l=[]

l = []
with open(to_open, encoding='utf-8') as f:
    for line in f:
        if line.startswith("> "):
            is_number = False
            flag = []
            for short in faults_to_include:
                if short in line:
                    thing = short
                    flag.append('yes')
                    make_new_fault()
                    print(thing)
                else:
                    flag.append('no')
        elif line.startswith("#"):
            pass
        else:
            is_number = True
            while is_number == True:
                if 'yes' in flag:
                    number_line()
                    
                else:
                    pass
                break
        save_line()

Black_Mountain_fault
Northern_Death_Valley_fault
Southern_Death_Valley_fault
Owl_Lake_fault
Ash_Hill_fault
Airport_Lake_fault
Hunter_Mountain_fault
Little_Lake_fault
Eastern_Little_Lake_north_north_east_cross_fault
Owens_Valley_fault
Panamint_Valley_fault
Independence_fault
Independence_Sierra_Nevada_fault


In [ ]:
#moves the files in the faults folder with the short names defined in want to the worldbuilder to be folder (wb_2b)
#now when I say move, it deletes the file in the fault folder, so it's not there anymore and has to be remade
fault_names = os.listdir(r"C:\Users\19049\coding_files\faults")
for fault in fault_names:
    for short in faults_to_include:
        if short in fault:
            move()
        else:
            pass

In [ ]:
#I"m thinking that now we can identify the faults we want out of the pile, we put them into another folder, which we can then run indescriminate code on, which I suppose the purpose of that code
#would be to convert the trace data to a worldbuilder file format? which I think daniel has.

In [ ]:
##northing is adjusted by subtracting 3870000, easting by subtracting 345000
for file in os.listdir(wb_2b):
    if file.endswith('.csv'):
        read = wb_2b+file
        df = pd.read_csv(read,
                header = 0,
                usecols=['Easting(m)', 'Northing(m)', 'Elevation(m)']
                )
        ar = df.to_numpy[:1, 0] - 345000
        thing = df['Easting(m)']-345000
        for each in df['Easting(m)'][1:]:
            new = each - 345000
            # print(new)
            
        # print(file)
        # print(thing)
        # print(file)
        # print(df.loc[:, ('Easting(m)', 'Northing(m)')], '\n')

In [ ]:
#so what this does is take the files that are csv files in wb_2b and returns the cartesian coordinate
#ready for conversion into worldbuilder files
#northing is adjusted by subtracting 3870000, easting by subtracting 345000
#What I want this damn thing to do is take the two values e and n, change em, then throw them into a new file
def get_coords(folder_2b):
    for file in os.listdir(folder_2b):
        if file.endswith('.csv'):
            print(file)
            open_up = folder_2b+file
            with open(open_up) as csvfile:
                spamreader = csv.reader(csvfile)
                spamdictreader = csv.DictReader(csvfile)
                #spamwriter = csv.writer(csvfile, fieldnames=['Easting(m)', 'Northing(m)'])
                for line in spamdictreader:
                    
                    #print(map(float, line['Easting(m)'])+', '+map(float, line['Northing(m)']))
                    print(line)
get_coords(wb_2b)

In [ ]:
def trace(x, y):
    

In [ ]:
#So basically what I want to do is take the trace data that we have and ignore the elevation, since we can do stuff with that later (like elevation based temperature changes). So for now jut get the trace
#data into the worldbuilder files with 90 degree dip
#below is Daniel stuff

In [ ]:
def segment_string(length, thickness, angle, top_truncation, coordinate, total_sections, segment_num, current_segment):

    '''
    Writes the string that defines segments of a slab to the world builder file

    When initializing a slab in Worldbuilder, you must specify the same number of segments for each section along strike
    of the slab, which is troublesome since some sections of slab are much longer than others. To get around this,
    the slab is initialized with the same number of segments as the section with the maximum number of segments. 
    Then, sections with less than this number of segments are assigned filler sections with lengths of 0 m until
    they reach the required number of segments

    length          = the length of a given segment
    thickness       = the thickness of a given segment
    angle           = the dip of a given segment
    coordinate      = the trench coordinate index coupled to the current section
    total_sections  = the number of sections making up the slab
    segment_num     = the number of segments making up each section
    current_segment = the current segment index of a section
    '''
 
    string_total = ''
    filler_segment_number = abs(len(thickness) - segment_num)
    if current_segment < filler_segment_number:
        if len(thickness[0]) > 1:
            # Here we proceed as above, adding the segment strings until we reach the end of the array
            string_total += '{"length":' + str(0.0) + ', "thickness":' + str([np.max(thickness), np.max(thickness)]) + ', "angle":' + str([0.0, 0.0]) + '},\n'# \
                           # ', "angle":' + str([0,0]) + '},\n'
            return string_total
        
        else:
            # Here we proceed as above, adding the segment strings until we reach the end of the array
            string_total += '{"length":' + str(0.0) + ', "thickness":' + str([np.max(thickness)]) + ', "angle":' + str([0.0, 0.0]) + '},\n'# \
                           # ', "angle":' + str([0,0]) + '},\n'
            return string_total
        
        # Now we have reached the end of the array, but need to fill out the amount of segments to reach the maximum
        # segment number. Add segments with lengths of 1m, thicknesses of 1m, and dips of 1 degree until this is the case.
    else:
        if current_segment != segment_num - 1:
            string_total = '{"length":' + str(length[current_segment - filler_segment_number]) + ', "thickness":' + str(thickness[current_segment - filler_segment_number]) + \
                            ', "angle":' + str(angle[current_segment - filler_segment_number]) + ', "top truncation":' + str(top_truncation) + '},\n'
            return string_total

        elif current_segment == (segment_num - 1) and coordinate == total_sections:
            string_total += '{"length":' + str(length[current_segment - filler_segment_number]) + ', "thickness":' + str(thickness[current_segment - filler_segment_number]) + \
                        ', "angle":' + str(angle[current_segment - filler_segment_number]) + ', "top truncation":' + str(top_truncation) + '}]}\n'
            return string_total

        else:
            string_total += '{"length":' + str(length[current_segment - filler_segment_number]) + ', "thickness":' + str(thickness[current_segment - filler_segment_number]) + \
                        ', "angle":' + str(angle[current_segment - filler_segment_number]) + ', "top truncation":' + str(top_truncation) + '}]},\n'
            return string_total

In [ ]:
def segment_section(world_builder_file, profile_directory, xshift, yshift, slab_thickness, top_truncation, coordinate_system):
    
    '''
    Calculates dip, thickness, and length of segments then uses segment_string() to create the string for the world builder file.
    This function has the option to vary the slab thickness between certain depths. If the varialbe slab_thickness is a scalar,
    a constant thickness is assumed for the entire section. To vary thickness, slab_thickness must be an array where each entry is an 
    array with length two. The entries of this array are the thickness of the slab and the depth to where 
    that thickness occurs. For example, to have the slab be 100km thick between 0 <= depth <= 200km, and then
    have the thickness increase to 150km for depth > 200km, the variable slab_thickness would need to be set as:

    slab_thickness = [ [100e3, 200e3], [150e3, 1e10] ]

    Where 1e10 was chosen to be a depth so high it would never be reached. Slab thickness and dip are varied gradually 
    down dip

    world_builder_file = the name of the world builder file
    profile_directory  = directory containing the world builder file
    xshift             = the amount of shift in the x direction, m
    yshift             = the amount of shift in the y direction, m
    slab_thickness     = the thickness of the slab, scaler for uniform thickness, array for variable, km
    '''
  
    # Import packages for spherical coordinate systems
    import geopy
    from geopy.distance import geodesic
    from geopy.distance import distance
    import pyproj
    geodesic = pyproj.Geod(ellps='WGS84')
    
    # Create an array which stores the length of each profile, the longest profile determines the number of
    # segments required for initializing the slab
    length = []
    for file in np.sort(os.listdir(profile_directory)):
        length.append(len(np.loadtxt(fname=profile_directory + file)))
    segment_num = max(length)
    total_sections = len(os.listdir(profile_directory)) - 1
    
    # This loop initializes the slab with the correct number of segments
    world_builder_file.write('"segments":[\n')
    for i in range(segment_num):
        if i != segment_num - 1:
            world_builder_file.write('{"length":0, "thickness":[0.0], "angle":[0]},\n')
        else:
            world_builder_file.write('{"length":0, "thickness":[0.0], "angle":[0]}],\n\n')

    # This loops through all sections and determines the thickness, dip, and length of each segment
    world_builder_file.write('    "sections":[')
    index = 0
    for file in np.sort(os.listdir(profile_directory)):
        index += 1
        filename = os.path.join(profile_directory, file)

        track_x = np.loadtxt(fname=filename, usecols=0) + xshift
        track_y = np.loadtxt(fname=filename, usecols=1) + yshift
        track_z = np.loadtxt(fname=filename, usecols=2)

        slab_length = 0
        segment_length = []
        dip_holder = [0]
        
        # Here we check to see if the thickness is set to vary along the slab by checking if slab_thickness is
        # a scalar or not. Dip is also computed and stored for output to the world builder file here
        if hasattr(slab_thickness, "__len__"):
            thick_holder = [slab_thickness[index - 1][:, 0][0]]
            for i in range(1, len(track_z)):
                thick_index = np.min(np.where( slab_thickness[index - 1][:, 1] > np.abs(track_z[i]) ))
                thick_holder.append(slab_thickness[index - 1][:, 0][thick_index])
                
                if coordinate_system == 'Cartesian':
                    slab_cart_proj = np.sqrt( (track_x[i] - track_x[i - 1])**2 + (track_y[i] - track_y[i - 1])**2 )
                    slab_length = np.sqrt( (track_x[i] - track_x[i - 1])**2 + (track_y[i] - track_y[i - 1])**2 + (track_z[i] - track_z[i - 1])**2)
                    riserun = (track_z[i - 1] - track_z[i]) / (slab_cart_proj)
                    
                elif coordinate_system == 'Spherical':
                    fwd_az, back_az, slab_cart_proj_m = geodesic.inv(track_x[i-1], track_y[i-1], track_x[i], track_y[i])
                    slab_cart_proj = slab_cart_proj_m / 1e3
                    slab_length = np.sqrt( slab_cart_proj**2 + (track_z[i] - track_z[i - 1])**2)
                    riserun = (track_z[i - 1] - track_z[i]) / (slab_cart_proj)
                    
                segment_length.append(slab_length * 1000)
                dip_holder.append(np.rad2deg(np.arctan(abs(riserun))))
                
            segment_thickness = []
            for k in range(len(thick_holder) - 1):
                if k != int(len(thick_holder) - 1):
                    segment_thickness.append([thick_holder[k], thick_holder[k + 1]])
                else:
                    segment_thickness.append([thick_holder[k]])
         
        else:
            segment_thickness = []
            for i in range(1, len(track_z)):
                segment_thickness.append([slab_thickness])
                
                if coordinate_system == 'Cartesian':
                    slab_cart_proj = np.sqrt( (track_x[i] - track_x[i - 1])**2 + (track_y[i] - track_y[i - 1])**2 )
                    slab_length = np.sqrt( (track_x[i] - track_x[i - 1])**2 + (track_y[i] - track_y[i - 1])**2 + (track_z[i] - track_z[i - 1])**2)
                    
                elif coordinate_system == 'Spherical':
                    fwd_az, back_az, slab_cart_proj_m = geodesic.inv(track_x[i-1], track_y[i-1], track_x[i], track_y[i])
                    slab_cart_proj = slab_cart_proj_m / 1e3
                    slab_length = np.sqrt( slab_cart_proj**2 + (track_z[i] - track_z[i - 1])**2)
                    
                riserun = (track_z[i - 1] - track_z[i]) / (slab_cart_proj)
                segment_length.append(slab_length * 1000)
                dip_holder.append(np.rad2deg(np.arctan(abs(riserun))))
                
        dips = []
        for k in range(len(dip_holder) - 1):
            if k != int(len(dip_holder) - 1):
                dips.append([dip_holder[k], dip_holder[k + 1]])
            else:
                dips.append([dip_holder[k]])
        
        # Write to the World Builder File
        world_builder_file.write('    {"coordinate":' + str(index - 1) + ',\n')
        world_builder_file.write('     "segments":[')
        for current_segment in range(segment_num):
            world_builder_file.write('    ' + segment_string(segment_length, segment_thickness, dips, top_truncation, index - 1, total_sections, segment_num, current_segment))